In [4]:
import json
import re
DATA_PATH = "C:/Users/utkar/Downloads/large_data.json"

In [11]:
day_dict = {
    "Saturdays": {},
    "Sundays": {},
    "Mondays": {},
    "Tuesdays": {},
    "Wednesdays": {},
    "Thursdays": {},
    "Fridays": {}
}
zipcode_restaurants = {}

# Initialize all the hours of the days
for d in day_dict.keys():
    print(d)
    for i in range(1,25):
        day_dict[d][i] = {}

Saturdays
Sundays
Mondays
Tuesdays
Wednesdays
Thursdays
Fridays


In [12]:
inputfile = open(DATA_PATH, "r")

#Read all the restaurant data
for line in inputfile:
    # Fetch the yelp ui
    restaurant = json.loads(line)
    
    # Parse Popular times data here
    regex = r'[0-9]*%'
    t = r'(?<=at).+'
    if 'uigoogle' in restaurant.keys():
        if restaurant['uigoogle'].get('popularTimes') and len(restaurant['uigoogle'].get('popularTimes')) > 0:
            # Fetch the zipcode
            addr_components = restaurant['google']['address_components']
            zipcode = None
            for comp in addr_components:
                if 'postal_code' in comp['types']:
                    zipcode = comp['long_name'].split('-')[0][:5]
                    break
            if zipcode == None:
                continue
        
            zipcode_restaurants[zipcode] = zipcode_restaurants.get(zipcode, 0) + 1
        
            time = restaurant['uigoogle']['popularTimes']
            # For a particular day
            for day in time:
                # Time zones
                for business in time[day]:
                    match = re.match(regex, business)
                    time_zone = re.search(t, business)
                    if match != None and time_zone != None:
                        busy = match.group(0).replace('%','')
                        ztime = time_zone.group(0).strip().split()
                        
                        if busy != 0 and len(busy) > 0:
                            clock = int(ztime[0]) + (0,12)[ztime[1].startswith('P')]
                            day_dict[day][clock][zipcode] = day_dict[day][clock].get(zipcode, 0) + int(busy)
            
inputfile.close()

In [20]:
for day in day_dict.keys():
    for hour in day_dict[day].keys():
        for zipcode in day_dict[day][hour].keys():
            day_dict[day][hour][zipcode] = round(day_dict[day][hour][zipcode]/zipcode_restaurants[zipcode], 2)

In [21]:
outputfile = open("C:/Users/utkar/Downloads/popular_times.json", "w")
outputfile.write(json.dumps(day_dict))
outputfile.close()